# PREMIER LEAGUE API

In [213]:
import requests, json
from pprint import pprint

import pandas as pd
pd.set_option('display.max_columns', None)

from tqdm.auto import tqdm
tqdm.pandas()

In [214]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

In [215]:
# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# get player data from 'elements' field
players = r['elements']

# create players dataframe
players = pd.json_normalize(r['elements'])

# create teams dataframe
teams = pd.json_normalize(r['teams'])

# 1. Players

## 1.1 Extract data

In [216]:
# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# get player data from 'elements' field
players = r['elements']

# create players dataframe
players = pd.json_normalize(r['elements'])

In [217]:
# inspect dimensions
players.shape

(685, 88)

In [218]:
# inspect columns
players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [219]:
players.to_clipboard()

## 1.2 Wrangling

In [220]:
# drop unnecessary columns
players = players.drop('dreamteam_count', axis=1)
players = players.drop('ep_next', axis=1)
players = players.drop('ep_this', axis=1)
players = players.drop('in_dreamteam', axis=1)
players = players.drop('special', axis=1)
players = players.drop('squad_number', axis=1)
players = players.drop('team_code', axis=1)
players = players.drop('influence_rank_type', axis=1)
players = players.drop('creativity_rank_type', axis=1)
players = players.drop('threat_rank_type', axis=1)
players = players.drop('expected_goals_per_90', axis=1)
players = players.drop('saves_per_90', axis=1)
players = players.drop('expected_assists_per_90', axis=1)
players = players.drop('expected_goal_involvements_per_90', axis=1)
players = players.drop('expected_goals_conceded_per_90', axis=1)
players = players.drop('goals_conceded_per_90', axis=1)
players = players.drop('now_cost_rank_type', axis=1)
players = players.drop('form_rank_type', axis=1)
players = players.drop('points_per_game_rank_type', axis=1)
players = players.drop('selected_rank_type', axis=1)
players = players.drop('starts_per_90', axis=1)
players = players.drop('clean_sheets_per_90', axis=1)
players = players.drop('code', axis=1)

In [221]:
# correct cost columns (divide by 10)
players['cost_change_event'] = players['cost_change_event']/10
players['cost_change_event_fall'] = players['cost_change_event_fall']/10
players['cost_change_start'] = players['cost_change_start']/10
players['cost_change_start_fall'] = players['cost_change_start_fall']/10
players['now_cost'] = players['now_cost']/10

In [222]:
# rename columns
players.rename(
    columns={'event_points' : 'round_points',
             'transfers_in_event' : 'transfers_in_round',
             'transfers_out_event' : 'transfers_out_round',
             'bonus' : 'bonus_points',
             'id' : 'player_id',
             'web_name' : 'display_name',
             'team' : 'team_id',
             'element_type' : 'position_id'
            },
    inplace=True)

In [223]:
# rename values in status column

# Create a dictionary to specify the value replacements
value_replace_dict = {'a': 'Available', 's': 'Suspended', 'i': 'Injured', 'd': 'Doubt', 'u': 'Unknown Return Date'}

# Use the replace() function to replace values in the 'status' column
players['status'] = players['status'].replace(value_replace_dict)

In [224]:
# create full name column
players['player_full_name'] = players['second_name']+', '+players['first_name']

In [225]:
players.to_clipboard()

# 2. Teams

## 2.1 Extract data

In [226]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

In [227]:
teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1230,1285,1250,1250,1210,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1115,1175,1130,1190,1100,1160,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1060,1095,1050,1100,1060,1090,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1125,1205,1120,1220,1130,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1165,1210,1120,1200,1210,1240,131


## 2.2 Wrangling

In [228]:
# drop unnecessary columns
teams = teams.drop('code', axis=1)
teams = teams.drop('draw', axis=1)
teams = teams.drop('form', axis=1)
teams = teams.drop('loss', axis=1)
teams = teams.drop('played', axis=1)
teams = teams.drop('points', axis=1)
teams = teams.drop('position', axis=1)
teams = teams.drop('strength', axis=1)
teams = teams.drop('team_division', axis=1)
teams = teams.drop('win', axis=1)
teams = teams.drop('strength_overall_home', axis=1)
teams = teams.drop('strength_overall_away', axis=1)
teams = teams.drop('strength_attack_home', axis=1)
teams = teams.drop('strength_attack_away', axis=1)
teams = teams.drop('strength_defence_home', axis=1)
teams = teams.drop('strength_defence_away', axis=1)
teams = teams.drop('pulse_id', axis=1)

In [229]:
# rename columns
teams.rename(columns=
                     {'id' : 'team_id',
                      'name' : 'team_name',
                      'short_name' : 'team_code',
                      'unavailable' : 'team_unavailable'
                     },
    inplace = True)

In [230]:
teams.head()

,team_id,team_name,team_code,team_unavailable
0,1,Arsenal,ARS,False
1,2,Aston Villa,AVL,False
2,3,Bournemouth,BOU,False
3,4,Brentford,BRE,False
4,5,Brighton,BHA,False


In [231]:
# join players to teams
df = players.merge(teams, on='team_id', how='left')

# 3. Positions

## 3.1 Extract data

In [232]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

In [233]:
# inspect shape
positions.shape

(4, 11)

In [234]:
# inspect positions
positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],81
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],222
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],294
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],88


## 3.2 Wrangling

In [235]:
# drop unnecessary columns
positions = positions.drop('plural_name', axis=1)
positions = positions.drop('singular_name_short', axis=1)
positions = positions.drop('squad_select', axis=1)
positions = positions.drop('squad_min_play', axis=1)
positions = positions.drop('squad_max_play', axis=1)
positions = positions.drop('ui_shirt_specific', axis=1)
positions = positions.drop('sub_positions_locked', axis=1)
positions = positions.drop('element_count', axis=1)

In [236]:
# rename columns
positions.rename(columns=
                 {'plural_name_short' : 'position_code',
                  'singular_name' : 'position',
                  'id' : 'position_id'
                 },
                 inplace=True)
                  

In [237]:
positions

,position_id,position_code,position
0,1,GKP,Goalkeeper
1,2,DEF,Defender
2,3,MID,Midfielder
3,4,FWD,Forward


In [238]:
# join positions to dataframe
df = df.merge(positions, on='position_id', how='left')

In [239]:
df = df.drop('position_id', axis = 1)

In [240]:
df.shape

(685, 70)

In [241]:
df.to_clipboard()

# 4. FDR

## 4.1 Extract data

In [242]:
# List to store player fixture dataframes
all_player_fixtures = []

# Iterate over player IDs
for player_id in range(1, df['player_id'].max()+1):
    # Get fixture data from element-summary endpoint
    r = requests.get(base_url + f'element-summary/{player_id}/').json()
    
    # Check if 'fixtures' field exists in the response
    if 'fixtures' in r:
        # Get fixture data from 'fixtures' field
        fixtures = r['fixtures']
        
        # Create player fixture dataframe
        df_fixtures = pd.json_normalize(fixtures)
        
        # Add player ID column to the dataframe
        df_fixtures['player_id'] = player_id
        
        # Append the player dataframe to the list
        all_player_fixtures.append(df_fixtures)
    else:
        print(f"No fixture data found for player ID {player_id}")

# Concatenate all player dataframes into one dataframe
final_fixtures = pd.concat(all_player_fixtures, ignore_index=True)

No fixture data found for player ID 612


In [243]:
# inspect fixtures
final_fixtures.head()

,id,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id
0,31,2367568,1,None,14,None,4.0,False,0,False,2023-09-03T15:30:00Z,Gameweek 4,True,3,1
1,43,2367580,9,None,1,None,5.0,False,0,False,2023-09-16T16:30:00Z,Gameweek 5,False,2,1
2,51,2367588,1,None,18,None,6.0,False,0,False,2023-09-24T13:00:00Z,Gameweek 6,True,3,1
3,62,2367598,3,None,1,None,7.0,False,0,False,2023-09-30T14:00:00Z,Gameweek 7,False,2,1
4,71,2367608,1,None,13,None,8.0,False,0,False,2023-10-08T15:30:00Z,Gameweek 8,True,5,1


In [244]:
final_fixtures.to_clipboard()

## 4.2 Wrangling

In [245]:
# drop unnecessary columns
final_fixtures = final_fixtures.drop('id', axis=1)
final_fixtures = final_fixtures.drop('code', axis=1)
final_fixtures = final_fixtures.drop('team_h_score', axis=1)
final_fixtures = final_fixtures.drop('team_a_score', axis=1)
final_fixtures = final_fixtures.drop('finished', axis=1)
final_fixtures = final_fixtures.drop('minutes', axis=1)
final_fixtures = final_fixtures.drop('provisional_start_time', axis=1)
final_fixtures = final_fixtures.drop('kickoff_time', axis=1)

In [246]:
# rename columns
final_fixtures.rename(columns=
                      {'team_h' : 'home_team_id',
                       'team_a' : 'away_team_id',
                       'event' : 'round',
                       'event_name' : 'gameweek_name',
                       'is_home' : 'is_home_game',
                       'difficult' : 'FDR'
                      },
                      inplace = True)

In [247]:
# create opposition column

# Define a function to apply the rule
def get_opposition_team_id(row):
    if row['is_home_game']:
        return row['away_team_id']
    else:
        return row['home_team_id']

# Apply the function to create the 'opposition_team_id' column
final_fixtures['opposition_team_id'] = final_fixtures.apply(get_opposition_team_id, axis=1)

In [248]:
# merge team name and codes to fixtures
final_fixtures = final_fixtures.merge(teams, left_on='opposition_team_id', right_on='team_id', how='left')

In [249]:
# rename columns
final_fixtures.rename(columns=
                      {'team_name' : 'opposition_team_name',
                       'team_code' : 'opposition_team_code'
                      },
                      inplace=True)

In [250]:
# drop unnecessary columns
final_fixtures = final_fixtures.drop('team_id', axis=1)
final_fixtures = final_fixtures.drop('team_unavailable', axis=1)

In [251]:
final_fixtures.head()

,home_team_id,away_team_id,round,gameweek_name,is_home_game,difficulty,player_id,opposition_team_id,opposition_team_name,opposition_team_code
0,1,14,4.0,Gameweek 4,True,3,1,14,Man Utd,MUN
1,9,1,5.0,Gameweek 5,False,2,1,9,Everton,EVE
2,1,18,6.0,Gameweek 6,True,3,1,18,Spurs,TOT
3,3,1,7.0,Gameweek 7,False,2,1,3,Bournemouth,BOU
4,1,13,8.0,Gameweek 8,True,5,1,13,Man City,MCI


In [252]:
final_fixtures.to_clipboard()

# FBREF DEPTH STATS

In [253]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
 
# instantiate options 
options = webdriver.ChromeOptions() 

In [254]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [255]:
from bs4 import BeautifulSoup

# 1. Shooting stats

## 1.1 Extract data

In [256]:

# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options)

# Specify the URL
url = 'https://fbref.com/en/comps/9/shooting/Premier-League-Stats'

# Get the entire website content
driver.get(url)

# Wait for the table container element to be visible
wait = WebDriverWait(driver, 10)
table_container = wait.until(EC.presence_of_element_located((By.ID, 'div_stats_shooting')))

# Extract the table content as HTML
table_html = table_container.get_attribute('innerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Find the table element
table = soup.find('table')

# Convert the HTML table to a DataFrame
shooting = pd.read_html(str(table))[0]

# Close the WebDriver
driver.quit()

C:\Users\muril\AppData\Local\Temp\ipykernel_17372\588382136.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [257]:
shooting.head()

Unnamed: 0_level_0  Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk              Player             Nation   
0                  1          Max Aarons            eng ENG   
1                  2  Bénie Adama Traore             ci CIV   
2                  3    Tosin Adarabioyo            eng ENG   
3                  4      Elijah Adebayo            eng ENG   
4                  5       Simon Adingra             ci CIV   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0                 DF        Bournemouth             23-236               2000   
1              FW,MF      Sheffield Utd             20-271               2002   
2                 DF             Fulham             25-338               1997   
3                 FW         Luton Town             25-233               1998   
4                 FW           Brighton             21-239               2002   

  Unnamed: 7_level_0 Standard                                                  \
                 90s      Gls Sh SoT   SoT% Sh/90 SoT/90  G/Sh G/SoT  Dist FK   
0                2.8        0  1   0    0.0  0.35   0.00  0.00   NaN  23.9  0   
1                2.9        0  1   1  100.0  0.35   0.35  0.00  0.00  15.3  0   
2                0.0        0  0   0    NaN  0.00   0.00   NaN   NaN   NaN  0   
3                1.5        0  0   0    NaN  0.00   0.00   NaN   NaN   NaN  0   
4                0.4        1  1   1  100.0  2.50   2.50  1.00  1.00   5.3  0   

           Expected                            Unnamed: 25_level_0  
  PK PKatt       xG npxG npxG/Sh  G-xG np:G-xG             Matches  
0  0     0      0.0  0.0    0.02   0.0     0.0             Matches  
1  0     0      0.3  0.3    0.27  -0.3    -0.3             Matches  
2  0     0      0.0  0.0     NaN   0.0     0.0             Matches  
3  0     0      0.0  0.0     NaN   0.0     0.0             Matches  
4  0     0      0.3  0.3    0.25  +0.7    +0.7             Matches

In [258]:
shooting.to_clipboard()

In [259]:
shooting.columns = shooting.columns.droplevel()

In [260]:
shooting.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,1,Max Aarons,eng ENG,DF,Bournemouth,23-236,2000,2.8,0,1,0,0.0,0.35,0.00,0.00,NaN,23.9,0,0,0,0.0,0.0,0.02,0.0,0.0,Matches
1,2,Bénie Adama Traore,ci CIV,"FW,MF",Sheffield Utd,20-271,2002,2.9,0,1,1,100.0,0.35,0.35,0.00,0.00,15.3,0,0,0,0.3,0.3,0.27,-0.3,-0.3,Matches
2,3,Tosin Adarabioyo,eng ENG,DF,Fulham,25-338,1997,0.0,0,0,0,NaN,0.00,0.00,NaN,NaN,NaN,0,0,0,0.0,0.0,NaN,0.0,0.0,Matches
3,4,Elijah Adebayo,eng ENG,FW,Luton Town,25-233,1998,1.5,0,0,0,NaN,0.00,0.00,NaN,NaN,NaN,0,0,0,0.0,0.0,NaN,0.0,0.0,Matches
4,5,Simon Adingra,ci CIV,FW,Brighton,21-239,2002,0.4,1,1,1,100.0,2.50,2.50,1.00,1.00,5.3,0,0,0,0.3,0.3,0.25,+0.7,+0.7,Matches


## 1.2 Wrangling

In [261]:
# inspect columns
shooting.dtypes

Rk         object
Player     object
Nation     object
Pos        object
Squad      object
Age        object
Born       object
90s        object
Gls        object
Sh         object
SoT        object
SoT%       object
Sh/90      object
SoT/90     object
G/Sh       object
G/SoT      object
Dist       object
FK         object
PK         object
PKatt      object
xG         object
npxG       object
npxG/Sh    object
G-xG       object
np:G-xG    object
Matches    object
dtype: object

In [262]:
# select variables of interest
shooting_red = shooting[['Player','SoT','SoT%','G/Sh']]

In [263]:
# remove redundant header rows
shooting_red = shooting_red[shooting_red.Player != 'Player']

In [264]:
# rename columns
shooting_red.rename(columns=
                      {'Player' : 'player',
                       'SoT' : 'shots_on_target',
                       'SoT%' : 'shots_on_target_percentage',
                       'G/Sh' : 'conversion_rate'
                      },
                      inplace = True)

## 1.3 Rank column derivation

In [265]:
# assign rankings to variables
columns = ['shots_on_target', 'shots_on_target_percentage',
       'conversion_rate']
for column in columns:
    shooting_red[column+'_rank'] = shooting_red[column].rank(method='min', ascending=False)

In [266]:
shooting_red.head()

,player,shots_on_target,shots_on_target_percentage,conversion_rate,shots_on_target_rank,shots_on_target_percentage_rank,conversion_rate_rank
0,Max Aarons,0,0.0,0.00,145.0,145.0,60.0
1,Bénie Adama Traore,1,100.0,0.00,69.0,113.0,60.0
2,Tosin Adarabioyo,0,NaN,NaN,145.0,NaN,NaN
3,Elijah Adebayo,0,NaN,NaN,145.0,NaN,NaN
4,Simon Adingra,1,100.0,1.00,69.0,113.0,1.0


# 2. Passing stats

## 2.1 Extract data

In [267]:

# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options)

# Specify the URL
url = 'https://fbref.com/en/comps/9/passing/Premier-League-Stats'

# Get the entire website content
driver.get(url)

# Wait for the table container element to be visible
wait = WebDriverWait(driver, 10)
table_container = wait.until(EC.presence_of_element_located((By.ID, 'div_stats_passing')))

# Extract the table content as HTML
table_html = table_container.get_attribute('innerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Find the table element
table = soup.find('table')

# Convert the HTML table to a DataFrame
passing = pd.read_html(str(table))[0]

# Close the WebDriver
driver.quit()

# Print the DataFrame
passing.head()


C:\Users\muril\AppData\Local\Temp\ipykernel_17372\3794907863.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


Unnamed: 0_level_0  Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk              Player             Nation   
0                  1          Max Aarons            eng ENG   
1                  2  Bénie Adama Traore             ci CIV   
2                  3    Tosin Adarabioyo            eng ENG   
3                  4      Elijah Adebayo            eng ENG   
4                  5       Simon Adingra             ci CIV   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0                 DF        Bournemouth             23-236               2000   
1              FW,MF      Sheffield Utd             20-271               2002   
2                 DF             Fulham             25-338               1997   
3                 FW         Luton Town             25-233               1998   
4                 FW           Brighton             21-239               2002   

  Unnamed: 7_level_0 Total                            Short           Medium  \
                 90s   Cmp  Att  Cmp% TotDist PrgDist   Cmp Att  Cmp%    Cmp   
0                2.8   114  143  79.7    1861     646    59  67  88.1     42   
1                2.9    31   42  73.8     483      99    18  21  85.7     12   
2                0.0     0    0   NaN       0       0     0   0   NaN      0   
3                1.5    20   25  80.0     314      78    11  13  84.6      8   
4                0.4    14   18  77.8     170      39    12  14  85.7      2   

            Long            Unnamed: 22_level_0 Unnamed: 23_level_0  \
  Att  Cmp%  Cmp Att   Cmp%                 Ast                 xAG   
0  52  80.8   10  14   71.4                   0                 0.1   
1  15  80.0    1   1  100.0                   0                 0.3   
2   0   NaN    0   0    NaN                   0                 0.0   
3  10  80.0    1   1  100.0                   0                 0.1   
4   3  66.7    0   0    NaN                   0                 0.1   

  Unnamed: 24_level_0 Unnamed: 25_level_0 Unnamed: 26_level_0  \
                   xA               A-xAG                  KP   
0                 0.2                -0.1                   2   
1                 0.2                -0.3                   2   
2                 0.0                 0.0                   0   
3                 0.1                -0.1                   2   
4                 0.0                -0.1                   1   

  Unnamed: 27_level_0 Unnamed: 28_level_0 Unnamed: 29_level_0  \
                  1/3                 PPA               CrsPA   
0                   7                   4                   2   
1                   1                   4                   0   
2                   0                   0                   0   
3                   1                   1                   0   
4                   0                   0                   0   

  Unnamed: 30_level_0 Unnamed: 31_level_0  
                 PrgP             Matches  
0                  11             Matches  
1                   6             Matches  
2                   0             Matches  
3                   5             Matches  
4                   0             Matches

In [268]:
passing.columns = passing.columns.droplevel()

## 2.2 Wrangling

In [269]:
# inspect columns
passing.dtypes

Rk         object
Player     object
Nation     object
Pos        object
Squad      object
Age        object
Born       object
90s        object
Cmp        object
Att        object
Cmp%       object
TotDist    object
PrgDist    object
Cmp        object
Att        object
Cmp%       object
Cmp        object
Att        object
Cmp%       object
Cmp        object
Att        object
Cmp%       object
Ast        object
xAG        object
xA         object
A-xAG      object
KP         object
1/3        object
PPA        object
CrsPA      object
PrgP       object
Matches    object
dtype: object

In [270]:
# select variables of interest
passing_red = passing[['Player','KP','1/3','PPA','CrsPA']]

In [271]:
# remove redundant header rows
passing_red = passing_red[passing_red.Player != 'Player']

In [272]:
# rename columns
passing_red.rename(columns=
                      {'Player' : 'player',
                       'KP' : 'key_passes',
                       '1/3' : 'passes_in_final_3rd',
                       'PPA' : 'passes_into_penalty_area',
                       'CrsPA' : 'crosses_into_penalty_area'
                      },
                      inplace = True)

## 2.3 Rank column derivation

In [273]:
# assign rankings to variables
columns = ['key_passes', 'passes_in_final_3rd',
       'passes_into_penalty_area', 'crosses_into_penalty_area']
for column in columns:
    passing_red[column+'_rank'] = passing_red[column].rank(method='min', ascending=False)

In [274]:
passing_red.head()

,player,key_passes,passes_in_final_3rd,passes_into_penalty_area,crosses_into_penalty_area,key_passes_rank,passes_in_final_3rd_rank,passes_into_penalty_area_rank,crosses_into_penalty_area_rank
0,Max Aarons,2,7,4,2,85.0,25.0,25.0,5.0
1,Bénie Adama Traore,2,1,4,0,85.0,248.0,25.0,68.0
2,Tosin Adarabioyo,0,0,0,0,221.0,290.0,195.0,68.0
3,Elijah Adebayo,2,1,1,0,85.0,248.0,126.0,68.0
4,Simon Adingra,1,0,0,0,143.0,290.0,195.0,68.0


# 3. Chance creation stats

## 3.1 Extract data

In [275]:

# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options)

# Specify the URL
url = 'https://fbref.com/en/comps/9/gca/Premier-League-Stats'

# Get the entire website content
driver.get(url)

# Wait for the table container element to be visible
wait = WebDriverWait(driver, 10)
table_container = wait.until(EC.presence_of_element_located((By.ID, 'div_stats_gca')))

# Extract the table content as HTML
table_html = table_container.get_attribute('innerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Find the table element
table = soup.find('table')

# Convert the HTML table to a DataFrame
chances = pd.read_html(str(table))[0]

# Close the WebDriver
driver.quit()

# Print the DataFrame
chances.head()


C:\Users\muril\AppData\Local\Temp\ipykernel_17372\1545838205.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


Unnamed: 0_level_0  Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk              Player             Nation   
0                  1          Max Aarons            eng ENG   
1                  2  Bénie Adama Traore             ci CIV   
2                  3    Tosin Adarabioyo            eng ENG   
3                  4      Elijah Adebayo            eng ENG   
4                  5       Simon Adingra             ci CIV   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0                 DF        Bournemouth             23-236               2000   
1              FW,MF      Sheffield Utd             20-271               2002   
2                 DF             Fulham             25-338               1997   
3                 FW         Luton Town             25-233               1998   
4                 FW           Brighton             21-239               2002   

  Unnamed: 7_level_0 SCA       SCA Types                        GCA        \
                 90s SCA SCA90  PassLive PassDead TO Sh Fld Def GCA GCA90   
0                2.8   7  2.48         4        1  0  0   2   0   0  0.00   
1                2.9   4  1.40         4        0  0  0   0   0   1  0.35   
2                0.0   0  0.00         0        0  0  0   0   0   0  0.00   
3                1.5   2  1.36         2        0  0  0   0   0   0  0.00   
4                0.4   1  2.50         1        0  0  0   0   0   0  0.00   

  GCA Types                        Unnamed: 24_level_0  
   PassLive PassDead TO Sh Fld Def             Matches  
0         0        0  0  0   0   0             Matches  
1         1        0  0  0   0   0             Matches  
2         0        0  0  0   0   0             Matches  
3         0        0  0  0   0   0             Matches  
4         0        0  0  0   0   0             Matches

In [276]:
# drop redundant index row
chances.columns = chances.columns.droplevel()

## 3.2 Wrangling

In [277]:
# select variables of interest
chances_red = chances[['Player','SCA']]

In [278]:
# remove redundant header rows
chances_red = chances_red[chances_red.Player != 'Player']

In [279]:
# rename columns
chances_red.rename(columns=
                      {'Player' : 'player',
                       'SCA' : 'chances_created'
                      },
                      inplace = True)

## 3.3 Rank column derivation

In [280]:
chances_red['chances_created_rank'] = chances_red['chances_created'].rank(method='min', ascending=False)

# 4. Defensive stats

## 4.1 Extract data

In [281]:

# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options)

# Specify the URL
url = 'https://fbref.com/en/comps/9/defense/Premier-League-Stats'

# Get the entire website content
driver.get(url)

# Wait for the table container element to be visible
wait = WebDriverWait(driver, 10)
table_container = wait.until(EC.presence_of_element_located((By.ID, 'div_stats_defense')))

# Extract the table content as HTML
table_html = table_container.get_attribute('innerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Find the table element
table = soup.find('table')

# Convert the HTML table to a DataFrame
defense = pd.read_html(str(table))[0]

# Close the WebDriver
driver.quit()

# Print the DataFrame
defense.head()


C:\Users\muril\AppData\Local\Temp\ipykernel_17372\3046601548.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


Unnamed: 0_level_0  Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk              Player             Nation   
0                  1          Max Aarons            eng ENG   
1                  2  Bénie Adama Traore             ci CIV   
2                  3    Tosin Adarabioyo            eng ENG   
3                  4      Elijah Adebayo            eng ENG   
4                  5       Simon Adingra             ci CIV   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0                 DF        Bournemouth             23-236               2000   
1              FW,MF      Sheffield Utd             20-271               2002   
2                 DF             Fulham             25-338               1997   
3                 FW         Luton Town             25-233               1998   
4                 FW           Brighton             21-239               2002   

  Unnamed: 7_level_0 Tackles                              Challenges      \
                 90s     Tkl TklW Def 3rd Mid 3rd Att 3rd        Tkl Att   
0                2.8       5    3       3       2       0          4   8   
1                2.9       2    1       0       1       1          0   4   
2                0.0       0    0       0       0       0          0   0   
3                1.5       0    0       0       0       0          0   0   
4                0.4       3    2       0       1       2          3   3   

              Blocks         Unnamed: 20_level_0 Unnamed: 21_level_0  \
    Tkl% Lost Blocks Sh Pass                 Int             Tkl+Int   
0   50.0    4      5  3    2                   0                   5   
1    0.0    4      2  0    2                   0                   2   
2    NaN    0      1  1    0                   0                   0   
3    NaN    0      1  0    1                   1                   1   
4  100.0    0      1  0    1                   0                   3   

  Unnamed: 22_level_0 Unnamed: 23_level_0 Unnamed: 24_level_0  
                  Clr                 Err             Matches  
0                   6                   0             Matches  
1                   0                   0             Matches  
2                   4                   0             Matches  
3                   3                   0             Matches  
4                   0                   0             Matches

In [282]:
# drop redundant index row
defense.columns = defense.columns.droplevel()

## 4.2 Wrangling

In [283]:
# inspect columns and datatypes
defense.dtypes

Rk         object
Player     object
Nation     object
Pos        object
Squad      object
Age        object
Born       object
90s        object
Tkl        object
TklW       object
Def 3rd    object
Mid 3rd    object
Att 3rd    object
Tkl        object
Att        object
Tkl%       object
Lost       object
Blocks     object
Sh         object
Pass       object
Int        object
Tkl+Int    object
Clr        object
Err        object
Matches    object
dtype: object

In [284]:
# select variables of interest
defense_red = defense[['Player','TklW','Blocks','Int','Clr']]

In [285]:
# remove redundant header rows
defense_red = defense_red[defense_red.Player != 'Player']

In [286]:
# rename columns
defense_red.rename(columns=
                      {'Player' : 'player',
                       'TklW' : 'dominant_tackles',
                       'Blocks' : 'blocks',
                       'Int' : 'interceptions',
                       'Clr' : 'clearances'
                      },
                      inplace = True)

## 4.3 Rank column derivation

In [287]:
# assign rankings to variables
columns = ['dominant_tackles', 'blocks', 'interceptions', 'clearances']
for column in columns:
    defense_red[column+'_rank'] = defense_red[column].rank(method='min', ascending=False)

In [288]:
defense_red.head()

,player,dominant_tackles,blocks,interceptions,clearances,dominant_tackles_rank,blocks_rank,interceptions_rank,clearances_rank
0,Max Aarons,3,5,0,6,57.0,20.0,197.0,27.0
1,Bénie Adama Traore,1,2,0,0,138.0,109.0,197.0,246.0
2,Tosin Adarabioyo,0,1,0,4,219.0,173.0,197.0,57.0
3,Elijah Adebayo,0,1,1,3,219.0,173.0,118.0,73.0
4,Simon Adingra,2,1,0,0,94.0,173.0,197.0,246.0


# 5. Possession stats

## 5.1 Extract data

In [289]:

# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options)

# Specify the URL
url = 'https://fbref.com/en/comps/9/possession/Premier-League-Stats'

# Get the entire website content
driver.get(url)

# Wait for the table container element to be visible
wait = WebDriverWait(driver, 10)
table_container = wait.until(EC.presence_of_element_located((By.ID, 'div_stats_possession')))

# Extract the table content as HTML
table_html = table_container.get_attribute('innerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Find the table element
table = soup.find('table')

# Convert the HTML table to a DataFrame
possession = pd.read_html(str(table))[0]

# Close the WebDriver
driver.quit()

# Print the DataFrame
possession.head()


C:\Users\muril\AppData\Local\Temp\ipykernel_17372\3638144149.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


Unnamed: 0_level_0  Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk              Player             Nation   
0                  1          Max Aarons            eng ENG   
1                  2  Bénie Adama Traore             ci CIV   
2                  3    Tosin Adarabioyo            eng ENG   
3                  4      Elijah Adebayo            eng ENG   
4                  5       Simon Adingra             ci CIV   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0                 DF        Bournemouth             23-236               2000   
1              FW,MF      Sheffield Utd             20-271               2002   
2                 DF             Fulham             25-338               1997   
3                 FW         Luton Town             25-233               1998   
4                 FW           Brighton             21-239               2002   

  Unnamed: 7_level_0 Touches                                               \
                 90s Touches Def Pen Def 3rd Mid 3rd Att 3rd Att Pen Live   
0                2.8     170       9      54      85      33       0  170   
1                2.9      67       1      10      28      31       7   67   
2                0.0       5       5       5       0       0       0    5   
3                1.5      36       3       4      20      13       2   36   
4                0.4      26       1       1       4      21       5   26   

  Take-Ons                       Carries                                       \
       Att Succ Succ% Tkld Tkld% Carries TotDist PrgDist PrgC 1/3 CPA Mis Dis   
0       10    4  40.0    3  30.0      79     535     276    3   5   0   1   0   
1       17    3  17.6   13  76.5      44     328     142    6   4   3   6   3   
2        0    0   NaN    0   NaN       0       0       0    0   0   0   0   0   
3        5    1  20.0    4  80.0      19     143      29    1   3   1   1   2   
4        4    1  25.0    3  75.0      15      81      36    2   0   3   1   0   

  Receiving      Unnamed: 30_level_0  
        Rec PrgR             Matches  
0        94    8             Matches  
1        40   11             Matches  
2         0    0             Matches  
3        23    7             Matches  
4        15    5             Matches

In [290]:
# drop redundant index row
possession.columns = possession.columns.droplevel()

In [291]:
possession.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Att,Succ,Succ%,Tkld,Tkld%,Carries,TotDist,PrgDist,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches
0,1,Max Aarons,eng ENG,DF,Bournemouth,23-236,2000,2.8,170,9,54,85,33,0,170,10,4,40.0,3,30.0,79,535,276,3,5,0,1,0,94,8,Matches
1,2,Bénie Adama Traore,ci CIV,"FW,MF",Sheffield Utd,20-271,2002,2.9,67,1,10,28,31,7,67,17,3,17.6,13,76.5,44,328,142,6,4,3,6,3,40,11,Matches
2,3,Tosin Adarabioyo,eng ENG,DF,Fulham,25-338,1997,0.0,5,5,5,0,0,0,5,0,0,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,Matches
3,4,Elijah Adebayo,eng ENG,FW,Luton Town,25-233,1998,1.5,36,3,4,20,13,2,36,5,1,20.0,4,80.0,19,143,29,1,3,1,1,2,23,7,Matches
4,5,Simon Adingra,ci CIV,FW,Brighton,21-239,2002,0.4,26,1,1,4,21,5,26,4,1,25.0,3,75.0,15,81,36,2,0,3,1,0,15,5,Matches


## 5.2 Wrangling

In [292]:
# inspect columns and datatypes
possession.dtypes

Rk         object
Player     object
Nation     object
Pos        object
Squad      object
Age        object
Born       object
90s        object
Touches    object
Def Pen    object
Def 3rd    object
Mid 3rd    object
Att 3rd    object
Att Pen    object
Live       object
Att        object
Succ       object
Succ%      object
Tkld       object
Tkld%      object
Carries    object
TotDist    object
PrgDist    object
PrgC       object
1/3        object
CPA        object
Mis        object
Dis        object
Rec        object
PrgR       object
Matches    object
dtype: object

In [293]:
# select variables of interest
possession_red = possession[['Player','Att 3rd','Att Pen','CPA']]

In [294]:
# remove redundant header rows
possession_red = possession_red[possession_red.Player != 'Player']

In [295]:
# rename columns
possession_red.rename(columns=
                      {'Player' : 'player',
                       'Att 3rd' : 'touches_in_attacking_3rd',
                       'Att Pen' : 'touches_in_penalty_area',
                       'CPA' : 'dribbles_into_penalty_area'
                      },
                      inplace = True)

## 5.3 Rank column derivation

In [296]:
# assign rankings to variables
columns = ['touches_in_attacking_3rd', 'touches_in_penalty_area',
       'dribbles_into_penalty_area']
for column in columns:
    possession_red[column+'_rank'] = possession_red[column].rank(method='min', ascending=False)

In [297]:
possession_red.head()

,player,touches_in_attacking_3rd,touches_in_penalty_area,dribbles_into_penalty_area,touches_in_attacking_3rd_rank,touches_in_penalty_area_rank,dribbles_into_penalty_area_rank
0,Max Aarons,33,0,0,167.0,283.0,131.0
1,Bénie Adama Traore,31,7,3,177.0,22.0,24.0
2,Tosin Adarabioyo,0,0,0,332.0,283.0,131.0
3,Elijah Adebayo,13,2,1,287.0,138.0,81.0
4,Simon Adingra,21,5,3,229.0,52.0,24.0


# 6. Goalkeeping stats

## 6.1 Extract data

In [298]:

# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install()), options=options)

# Specify the URL
url = 'https://fbref.com/en/comps/9/keepers/Premier-League-Stats'

# Get the entire website content
driver.get(url)

# Wait for the table container element to be visible
wait = WebDriverWait(driver, 10)
table_container = wait.until(EC.presence_of_element_located((By.ID, 'div_stats_keeper')))

# Extract the table content as HTML
table_html = table_container.get_attribute('innerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Find the table element
table = soup.find('table')

# Convert the HTML table to a DataFrame
keeper = pd.read_html(str(table))[0]

# Close the WebDriver
driver.quit()

# drop redundant index row
keeper.columns = keeper.columns.droplevel()

# Print the DataFrame
keeper.head()


C:\Users\muril\AppData\Local\Temp\ipykernel_17372\1274570507.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKatt,PKA,PKsv,PKm,Save%,Matches
0,1,Alisson,br BRA,GK,Liverpool,30-330,1992,3,3,270,3.0,3,1.00,17,14,82.4,2,1,0,0,0.0,0,0,0,0,NaN,Matches
1,2,Alphonse Areola,fr FRA,GK,West Ham,30-182,1993,3,3,270,3.0,3,1.00,19,15,84.2,2,1,0,0,0.0,1,0,1,0,100.0,Matches
2,3,Ederson,br BRA,GK,Manchester City,30-011,1993,3,3,270,3.0,1,0.33,3,2,66.7,3,0,0,2,66.7,0,0,0,0,NaN,Matches
3,4,Mark Flekken,nl NED,GK,Brentford,30-076,1993,3,3,270,3.0,3,1.00,13,10,76.9,1,2,0,1,33.3,0,0,0,0,NaN,Matches
4,5,Wes Foderingham,eng ENG,GK,Sheffield Utd,32-226,1991,3,3,270,3.0,5,1.67,21,16,76.2,0,0,3,0,0.0,1,0,0,1,NaN,Matches


## 6.2 Wrangling

In [299]:
# inspect columns and datatypes
keeper.dtypes

Rk           int64
Player      object
Nation      object
Pos         object
Squad       object
Age         object
Born         int64
MP           int64
Starts       int64
Min          int64
90s        float64
GA           int64
GA90       float64
SoTA         int64
Saves        int64
Save%      float64
W            int64
D            int64
L            int64
CS           int64
CS%        float64
PKatt        int64
PKA          int64
PKsv         int64
PKm          int64
Save%      float64
Matches     object
dtype: object

In [300]:
# rename penalty save percentage column to avoid duplicate
keeper.columns.values[25] = 'PKSave%'

In [301]:
keeper.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts',
       'Min', '90s', 'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L',
       'CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'PKSave%', 'Matches'],
      dtype='object')

In [302]:
# select variables of interest
keeper_red = keeper[['Player','MP','Saves','Save%','CS%','PKsv','PKSave%']]

In [303]:
# rename columns
keeper_red.rename(columns=
                      {'MP' : 'matches_played',
                       'Saves' : 'saves',
                       'Save%' : 'save_percentage',
                       'CS%' : 'clean_sheet_percentage',
                       'PKsv' : 'penalties_saved',
                       'PKSave%' : 'penalty_save_percentage',
                       'Player' : 'player'
                      },
                      inplace = True)

C:\Users\muril\AppData\Local\Temp\ipykernel_17372\3031439251.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_red.rename(columns=


In [304]:
keeper_red.head()

,player,matches_played,saves,save_percentage,clean_sheet_percentage,penalties_saved,penalty_save_percentage
0,Alisson,3,14,82.4,0.0,0,NaN
1,Alphonse Areola,3,15,84.2,0.0,1,100.0
2,Ederson,3,2,66.7,66.7,0,NaN
3,Mark Flekken,3,10,76.9,33.3,0,NaN
4,Wes Foderingham,3,16,76.2,0.0,0,NaN


## 6.3 Rank column derivation

In [305]:
# assign rankings to variables
columns = ['matches_played', 'saves',
       'save_percentage','clean_sheet_percentage',
       'penalties_saved', 'penalty_save_percentage']
for column in columns:
    keeper_red[column+'_rank'] = keeper_red[column].rank(method='min', ascending=False)

C:\Users\muril\AppData\Local\Temp\ipykernel_17372\60635240.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_red[column+'_rank'] = keeper_red[column].rank(method='min', ascending=False)
C:\Users\muril\AppData\Local\Temp\ipykernel_17372\60635240.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_red[column+'_rank'] = keeper_red[column].rank(method='min', ascending=False)
C:\Users\muril\AppData\Local\Temp\ipykernel_17372\60635240.py:6: SettingWithCopyWarning: 
A value is trying to be set on a

In [306]:
keeper_red.head()

,player,matches_played,saves,save_percentage,clean_sheet_percentage,penalties_saved,penalty_save_percentage,matches_played_rank,saves_rank,save_percentage_rank,clean_sheet_percentage_rank,penalties_saved_rank,penalty_save_percentage_rank
0,Alisson,3,14,82.4,0.0,0,NaN,1.0,4.0,5.0,11.0,3.0,NaN
1,Alphonse Areola,3,15,84.2,0.0,1,100.0,1.0,3.0,4.0,11.0,1.0,1.0
2,Ederson,3,2,66.7,66.7,0,NaN,1.0,21.0,14.0,1.0,3.0,NaN
3,Mark Flekken,3,10,76.9,33.3,0,NaN,1.0,11.0,7.0,4.0,3.0,NaN
4,Wes Foderingham,3,16,76.2,0.0,0,NaN,1.0,2.0,8.0,11.0,3.0,NaN


# 7. Merge Preparation

In [307]:
# export player list
shooting_red['player'].to_clipboard()

In [308]:
# import player id's
players_id = pd.read_csv(r"C:\Users\muril\Data Analysis Projects\08-2023 FPL\02 Data\Prepared Data\FBREF Players ID.csv")

In [309]:
# rename to player_id
players_id.rename(columns={'id':'player_id'}, inplace=True)

In [310]:
players_id.head()

,player_id,player
0,372,Antony
1,159,Benson Manuel
2,334,Bernardo Silva
3,270,Bobby Reid
4,376,Casemiro


In [311]:
# merge players_id to datasets

In [312]:
shooting_red = shooting_red.merge(players_id, on='player', how='left')

In [313]:
passing_red = passing_red.merge(players_id, on='player', how='left')

In [314]:
chances_red = chances_red.merge(players_id, on='player', how='left')

In [315]:
defense_red = defense_red.merge(players_id, on='player', how='left')

In [316]:
possession_red = possession_red.merge(players_id, on='player', how='left')

In [317]:
keeper_red = keeper_red.merge(players_id, on='player', how='left')

In [318]:
# drop player name from each dataframe to avoid duplication on merge to main dataframe
shooting_red = shooting_red.drop('player', axis=1)
passing_red = passing_red.drop('player', axis=1)
chances_red = chances_red.drop('player', axis=1)
defense_red = defense_red.drop('player', axis=1)
possession_red = possession_red.drop('player', axis=1)
keeper_red = keeper_red.drop('player', axis=1)

# FULL DATASET MERGE

In [319]:
# merge depth dataframes to main dataframe
df_merged = df.merge(shooting_red, on='player_id', how='left')
df_merged = df_merged.merge(passing_red, on='player_id', how='left')
df_merged = df_merged.merge(chances_red, on='player_id', how='left')
df_merged = df_merged.merge(defense_red, on='player_id', how='left')
df_merged = df_merged.merge(possession_red, on='player_id', how='left')
df_merged = df_merged.merge(keeper_red, on='player_id', how='left')

In [320]:
###############

# RADAR CHART PREPARATION

## 1. Attacking measures

In [321]:
# select attacking variables for radar chart
att_variables = [
             'passes_in_final_3rd',    
             'crosses_into_penalty_area',
                'key_passes',
             'assists',
             'shots_on_target',
             'shots_on_target_percentage',
                 'goals_scored',
             'conversion_rate',
             'touches_in_attacking_3rd',
                 'touches_in_penalty_area',
             'dribbles_into_penalty_area'
]


In [322]:
# take subset for radar chart attacking variables
rad_var_att = df_merged[['player_id',
                        'passes_in_final_3rd',    
             'crosses_into_penalty_area',
                'key_passes',
             'assists',
             'shots_on_target',
             'shots_on_target_percentage',
                 'goals_scored',
             'conversion_rate',
             'touches_in_attacking_3rd',
                 'touches_in_penalty_area',
             'dribbles_into_penalty_area']]

In [323]:
# select variables for relative column derviation
variables = ['passes_in_final_3rd',    
             'crosses_into_penalty_area',
                'key_passes',
             'assists',
             'shots_on_target',
                 'goals_scored',
             'touches_in_attacking_3rd',
                 'touches_in_penalty_area',
             'dribbles_into_penalty_area']

In [324]:
# replace nulls with 0
rad_var_att.fillna(0,inplace=True)

C:\Users\muril\AppData\Local\Temp\ipykernel_17372\2605315599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rad_var_att.fillna(0,inplace=True)


In [325]:
# change all datatypes to decimals
rad_var_att = rad_var_att.astype(float)

In [326]:
# create relative variable columns
for var in variables:
            rad_var_att[var + '_relative'] = rad_var_att[var] / rad_var_att[var].max()

In [327]:
rad_var_att['shots_on_target_percentage_relative'] = rad_var_att['shots_on_target_percentage']/100 

In [328]:
att_rel = rad_var_att[['player_id','passes_in_final_3rd_relative',
       'crosses_into_penalty_area_relative', 'key_passes_relative',
       'assists_relative', 'shots_on_target_relative','shots_on_target_percentage_relative', 'goals_scored_relative', 'conversion_rate',
       'touches_in_attacking_3rd_relative', 'touches_in_penalty_area_relative',
       'dribbles_into_penalty_area_relative']]
       

In [329]:
att_rel

,player_id,passes_in_final_3rd_relative,crosses_into_penalty_area_relative,key_passes_relative,assists_relative,shots_on_target_relative,shots_on_target_percentage_relative,goals_scored_relative,conversion_rate,touches_in_attacking_3rd_relative,touches_in_penalty_area_relative,dribbles_into_penalty_area_relative
0,1.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
1,2.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
2,3.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
3,4.0,0.000000,0.00,0.000000,0.666667,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
4,5.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
680,572.0,0.400000,0.25,0.071429,0.000000,0.000000,0.00,0.0,0.0,0.198795,0.030303,0.000000
681,573.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
682,574.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000
683,590.0,0.228571,0.00,0.285714,0.000000,0.285714,0.25,0.0,0.0,0.216867,0.242424,0.153846


In [330]:
# Melt the DataFrame to unpivot it
att_rel_unpivoted = pd.melt(att_rel, id_vars=['player_id'], value_vars=['passes_in_final_3rd_relative',
       'crosses_into_penalty_area_relative', 'key_passes_relative',
       'assists_relative', 'shots_on_target_relative','shots_on_target_percentage_relative', 'goals_scored_relative', 'conversion_rate',
       'touches_in_attacking_3rd_relative', 'touches_in_penalty_area_relative',
       'dribbles_into_penalty_area_relative'],
                       var_name='attack_measure', value_name='attack_relative_score')

In [331]:
att_rel_unpivoted.shape

(7535, 3)

In [332]:
att_rel_unpivoted.to_clipboard()

# 2. Defensive measures

In [333]:
# select variables for defensive radar chart
def_variables = ['dominant_tackles', 'blocks', 'interceptions', 'clearances','player_id',
                 'saves', 'save_percentage', 'clean_sheet_percentage','penalties_saved', 
                 'penalty_save_percentage', 'clean_sheets']

In [334]:
df_merged.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'round_points', 'first_name', 'form',
       'player_id',
       ...
       'save_percentage', 'clean_sheet_percentage', 'penalties_saved_y',
       'penalty_save_percentage', 'matches_played_rank', 'saves_rank',
       'save_percentage_rank', 'clean_sheet_percentage_rank',
       'penalties_saved_rank', 'penalty_save_percentage_rank'],
      dtype='object', length=112)

In [335]:
# take subset for radar chart attacking variables
rad_var_def = df_merged[['dominant_tackles', 'blocks', 'interceptions', 'clearances','player_id',
                 'saves_x', 'save_percentage', 'clean_sheet_percentage','penalties_saved_x', 
                 'penalty_save_percentage', 'clean_sheets']]

In [336]:
# replace nulls with 0
rad_var_def.fillna(0,inplace=True)

C:\Users\muril\AppData\Local\Temp\ipykernel_17372\1509559136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rad_var_def.fillna(0,inplace=True)


In [337]:
# change all datatypes to decimals
rad_var_def = rad_var_def.astype(float)

In [338]:
rad_var_def.dtypes

dominant_tackles           float64
blocks                     float64
interceptions              float64
clearances                 float64
player_id                  float64
saves_x                    float64
save_percentage            float64
clean_sheet_percentage     float64
penalties_saved_x          float64
penalty_save_percentage    float64
clean_sheets               float64
dtype: object

In [339]:
# select variables for relative variable derivation
variables = ['dominant_tackles', 'blocks', 'interceptions', 'clearances',
                 'saves_x','penalties_saved_x', 'clean_sheets']

In [340]:
# create relative variable columns
for var in variables:
            rad_var_def[var + '_relative'] = rad_var_def[var] / rad_var_def[var].max()

In [341]:
rad_var_def['save_percentage_relative'] = rad_var_def['save_percentage']/100

In [342]:
rad_var_def['clean_sheet_percentage_relative'] = rad_var_def['clean_sheet_percentage']/100

In [343]:
rad_var_def['penalty_save_percentage_relative'] = rad_var_def['penalty_save_percentage']/100

In [344]:
def_rel = rad_var_def[['player_id','dominant_tackles_relative', 'blocks_relative',
       'interceptions_relative', 'clearances_relative', 'saves_x_relative',
       'penalties_saved_x_relative', 'clean_sheets_relative',
       'save_percentage_relative', 'clean_sheet_percentage_relative',
       'penalty_save_percentage_relative']]

In [345]:
def_rel

,player_id,dominant_tackles_relative,blocks_relative,interceptions_relative,clearances_relative,saves_x_relative,penalties_saved_x_relative,clean_sheets_relative,save_percentage_relative,clean_sheet_percentage_relative,penalty_save_percentage_relative
0,1.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.090909,0.181818,0.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
680,572.0,0.636364,0.272727,0.4,0.28,0.0,0.0,0.5,0.0,0.0,0.0
681,573.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
682,574.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
683,590.0,0.181818,0.181818,0.2,0.08,0.0,0.0,0.5,0.0,0.0,0.0


In [346]:
# Melt the DataFrame to unpivot it
def_rel_unpivoted = pd.melt(def_rel, id_vars=['player_id'], value_vars=['dominant_tackles_relative', 'blocks_relative',
       'interceptions_relative', 'clearances_relative', 'saves_x_relative',
       'penalties_saved_x_relative', 'clean_sheets_relative',
       'save_percentage_relative', 'clean_sheet_percentage_relative',
       'penalty_save_percentage_relative'],
                       var_name='defense_measure', value_name='defense_relative_score')

In [347]:
def_rel_unpivoted.shape

(6850, 3)

In [348]:
def_rel_unpivoted.to_clipboard()

In [349]:
####################################

In [357]:
# copy to clipboard to export to google sheet, connected to dashboard
df_merged.to_clipboard()

In [351]:
####################################

In [358]:
att_rel_unpivoted.to_clipboard()

In [353]:
####################################

In [359]:
def_rel_unpivoted.to_clipboard()

In [355]:
####################################

In [356]:
# after exporting df_merged, now copy and export FDR to gsheet for dashboard (as required)